# Grover's Algorithm

**Companion notebook for:** [`grover.md`](./grover.md)

This notebook demonstrates:
- Grover's search algorithm providing quadratic speedup
- Amplitude amplification mechanics
- The oracle and diffusion operator
- Geometric interpretation on Bloch-like sphere

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram
from qiskit_aer import Aer

np.set_printoptions(precision=3, suppress=True)

: 

## 1. The Problem - Unstructured Search

Find a marked item in a list of N items.  
Classical: O(N) queries  
Grover: O(√N) queries

In [ ]:
# Example: search space of 2^3 = 8 items
n_qubits = 3
N = 2**n_qubits

# Let's say the marked item is |101⟩ = 5
marked_item = 5

print(f"Search space: {N} items")
print(f"Marked item: |{format(marked_item, f'0{n_qubits}b')}⟩ = {marked_item}")
print(f"\nClassical queries needed (worst case): {N}")
print(f"Grover iterations needed: ~{int(np.sqrt(N))}")
print(f"\nSpeedup: {N / np.sqrt(N):.1f}x")

## 2. Building the Oracle

The oracle marks the correct answer by flipping its phase.

In [ ]:
def create_oracle(n_qubits, marked_item):
    """
    Create an oracle that flips the phase of the marked item.
    For item |w⟩, applies O|w⟩ = -|w⟩ and O|x⟩ = |x⟩ for x ≠ w.
    """
    qc = QuantumCircuit(n_qubits)
    
    # Convert marked item to binary string
    marked_binary = format(marked_item, f'0{n_qubits}b')
    
    # Flip qubits that should be 0 in the marked state
    for qubit, bit in enumerate(marked_binary[::-1]):
        if bit == '0':
            qc.x(qubit)
    
    # Multi-controlled Z gate (phase flip)
    qc.h(n_qubits - 1)
    qc.mcx(list(range(n_qubits - 1)), n_qubits - 1)
    qc.h(n_qubits - 1)
    
    # Undo the X gates
    for qubit, bit in enumerate(marked_binary[::-1]):
        if bit == '0':
            qc.x(qubit)
    
    return qc

# Test the oracle
oracle = create_oracle(3, 5)
print("Oracle circuit:")
print(oracle.draw(output='text'))

## 3. The Diffusion Operator

After marking, we amplify the amplitude of the marked item.

In [ ]:
def create_diffusion(n_qubits):
    """
    Create the diffusion operator (inversion about average).
    This is 2|s⟩⟨s| - I where |s⟩ is the equal superposition.
    """
    qc = QuantumCircuit(n_qubits)
    
    # Apply H to all qubits
    qc.h(range(n_qubits))
    
    # Apply X to all qubits
    qc.x(range(n_qubits))
    
    # Multi-controlled Z
    qc.h(n_qubits - 1)
    qc.mcx(list(range(n_qubits - 1)), n_qubits - 1)
    qc.h(n_qubits - 1)
    
    # Apply X to all qubits
    qc.x(range(n_qubits))
    
    # Apply H to all qubits
    qc.h(range(n_qubits))
    
    return qc

diffusion = create_diffusion(3)
print("Diffusion operator:")
print(diffusion.draw(output='text'))

## 4. Complete Grover's Algorithm

In [ ]:
def grovers_algorithm(n_qubits, marked_item, iterations=None):
    """
    Complete Grover's search algorithm.
    """
    qc = QuantumCircuit(n_qubits, n_qubits)
    
    # Step 1: Create equal superposition
    qc.h(range(n_qubits))
    
    # Step 2: Apply Grover iterations
    if iterations is None:
        iterations = int(np.pi / 4 * np.sqrt(2**n_qubits))
    
    oracle = create_oracle(n_qubits, marked_item)
    diffusion = create_diffusion(n_qubits)
    
    for _ in range(iterations):
        qc.compose(oracle, inplace=True)
        qc.compose(diffusion, inplace=True)
    
    # Step 3: Measure
    qc.measure(range(n_qubits), range(n_qubits))
    
    return qc, iterations

# Run Grover's algorithm
qc, iters = grovers_algorithm(3, 5)

print(f"Running Grover's algorithm to find |101⟩ = 5")
print(f"Using {iters} Grover iterations")

# Simulate
backend = Aer.get_backend('qasm_simulator')
job = backend.run(transpile(qc, backend), shots=1000)
counts = job.result().get_counts()

plot_histogram(counts)
plt.title(f'Grover Search Results ({iters} iterations)')
plt.show()

print("\nMeasurement results:", counts)
best_result = max(counts.items(), key=lambda x: x[1])
print(f"\nMost frequent outcome: |{best_result[0]}⟩ = {int(best_result[0], 2)}")
print(f"Success probability: {best_result[1]/1000:.1%}")

## 5. Amplitude Evolution

Watch how the amplitude grows with each iteration.

In [ ]:
# Track amplitude evolution
n_qubits = 3
marked = 5
max_iters = 5

probabilities = []

for num_iters in range(max_iters + 1):
    qc = QuantumCircuit(n_qubits)
    qc.h(range(n_qubits))
    
    oracle = create_oracle(n_qubits, marked)
    diffusion = create_diffusion(n_qubits)
    
    for _ in range(num_iters):
        qc.compose(oracle, inplace=True)
        qc.compose(diffusion, inplace=True)
    
    state = Statevector(qc)
    prob = state.probabilities()[marked]
    probabilities.append(prob)

# Plot evolution
plt.figure(figsize=(10, 6))
plt.plot(range(max_iters + 1), probabilities, 'bo-', linewidth=2, markersize=8)
plt.axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Perfect amplification')
plt.xlabel('Number of Grover Iterations')
plt.ylabel('Probability of Marked Item')
plt.title('Amplitude Amplification Over Iterations')
plt.grid(True, alpha=0.3)
plt.legend()
plt.ylim([0, 1.1])
plt.show()

print("Probability after each iteration:")
for i, prob in enumerate(probabilities):
    print(f"  Iteration {i}: {prob:.4f}")

print("\n→ Probability grows quadratically")
print("→ Too many iterations causes it to decrease!")
print("→ Optimal iterations: ~√N")

## 6. Geometric Interpretation

Grover's algorithm rotates the state vector toward the solution.

In [ ]:
# Simplified geometric view
N = 8  # Search space
theta_per_iteration = 2 * np.arcsin(1 / np.sqrt(N))

print("Geometric picture of Grover's:")
print(f"\nSearch space: {N} items")
print(f"Initial angle from |w⟩: {np.pi/2:.4f} rad ({90:.1f}°)")
print(f"Rotation per iteration: {theta_per_iteration:.4f} rad ({np.degrees(theta_per_iteration):.1f}°)")

# How many iterations to reach |w⟩?
optimal_iters = int(np.pi / (4 * np.arcsin(1 / np.sqrt(N))))
print(f"\nOptimal iterations: {optimal_iters}")
print(f"Final angle: ~{optimal_iters * theta_per_iteration:.4f} rad")
print(f"This should be close to 0 (pointing at |w⟩)")

print("\n→ Each Grover iteration rotates by a fixed angle")
print("→ The algorithm 'walks' toward the solution geometrically")

## Summary

From this notebook, you should understand:

1. **Quadratic speedup** — O(√N) vs O(N) for unstructured search
2. **Oracle marks** — flips phase of the correct answer
3. **Diffusion amplifies** — inversion about average increases amplitude
4. **Geometric rotation** — state vector rotates toward solution
5. **Optimal iterations** — too few or too many both fail

Grover's is not exponential speedup, but it's:
- **Provably optimal** for unstructured search
- **General purpose** works for any search problem
- **Geometric** easy to visualize

This completes the notebook series!